# Мнемотехническая оптимизация

**Цель**: подобрать оптимальное соответствие пар согласных русского языка и цифр.

Представим следующую задачу: мы хотим разработать систему мнемотехнических правил для запоминания длинных чисел через их разбиение на группы по 2-3 цифры и сопоставление таким буквам слов, содержащих 2-3 согласных, которыми мы кодируем абстрактные цифры. Мы хотим оптимизировать выбор таких кодирующих букв с учетом частотности букв и существительных и равномерном распределении цифр.

Разделим буквы на глассные и согласные.

In [121]:
vows = "аиеёоуыэюя"
cons = "бвгджзйклмнпрстфхцчшщ"

In [122]:
len(cons)

21

В русском языке 21 согласная. Получается, что каждой цифре мы можем сопоставить даже не одну, а две буквы. При этом нужно понимать, что в нашей задаче мы не собираемся сопоставлять звучание или форму самих букв определенным символам (например, 4 - ч). Мы хотим построить таблицу без избыточного упоминания согласных.

## Частотный словарь

Для нашей задачи нам потребуется список словарных слов, в котором есть разделение слов по частям речи и желательно — оценка ранга частоты их встречаемости в русскоязычных текстах.

In [123]:
import pandas as pd
df = pd.read_csv('freq.csv', sep='\t')

In [124]:
df

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Выделим существительные:

In [125]:
dfs = df.loc[df['PoS'] == 's']

# сразу удалим все составные слова, содержащие дефис
dfs = dfs[dfs["Lemma"].str.contains('-')==False]
dfs

,Lemma,PoS,Freq(ipm),R,D,Doc
3,а,s,2.7,59,85,160
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
11,аббатство,s,0.9,33,77,40
12,аббревиатура,s,3.5,83,91,239
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Для описаннной задачи разбиения в 2-3 символа нам необходимы двух и трехбуквенные соответствия, то есть всего лишь порядка 1000 слов. Однако среди словарных нас в первую очередь интересуют те, что содержат 2 или 3 согласных, которые могут быть окружены любым количеством гласных. Опишем фильтр с помощью соответствующего регулярного выражения.

In [126]:
import re
pattern = re.compile(fr"([{vows}]*[{cons}]){{2,3}}[{vows}]*\b")
dfs23 = dfs[dfs.Lemma.str.match(pattern)]
dfs23

,Lemma,PoS,Freq(ipm),R,D,Doc
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
13,абвер,s,2.7,22,38,25
16,абзац,s,10.2,99,87,478
24,аборт,s,8.3,93,87,300
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Теперь мы можем сопоставить каждому слову числовой код и сравнить, при каком коде происходит наиболее полное покрытие. Для упрощения кодирования добавим столбец, в котором в словах оставлены одни согласные. Часть строк при этом начнет совпадать, так как в таком дополнительном столбце теряется информация о гласных.

In [127]:
dfs23['cons'] = dfs23.Lemma.str.replace(fr'[{vows}]', '', regex=True)

/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_84007/3844718368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23['cons'] = dfs23.Lemma.str.replace(fr'[{vows}]', '', regex=True)


In [128]:
dfs23.drop_duplicates(subset=['cons'], inplace=True)

/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_84007/2660652694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23.drop_duplicates(subset=['cons'], inplace=True)


In [129]:
dfs23

,Lemma,PoS,Freq(ipm),R,D,Doc,cons
9,абажур,s,4.9,79,90,269,бжр
10,аббат,s,1.6,44,79,63,ббт
13,абвер,s,2.7,22,38,25,бвр
16,абзац,s,10.2,99,87,478,бзц
24,аборт,s,8.3,93,87,300,брт
...,...,...,...,...,...,...,...
52056,ямщик,s,1.4,50,81,74,мщк
52119,ятаган,s,0.6,24,75,37,тгн
52132,ящер,s,2.0,51,73,63,щр
52133,ящерица,s,3.6,77,74,158,щрц


Проверим, что в стобце есть только 2 или 3 согласных.

In [132]:
dfs23['cons'].apply(len).describe()

count    2253.000000
mean        2.859299
std         0.347790
min         2.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         3.000000
Name: cons, dtype: float64

Проверим, что сочетания согласных уникальны:

In [133]:
dfs23['cons'].describe()

count     2253
unique    2253
top        бжр
freq         1
Name: cons, dtype: object

In [134]:
dfs23

,Lemma,PoS,Freq(ipm),R,D,Doc,cons
9,абажур,s,4.9,79,90,269,бжр
10,аббат,s,1.6,44,79,63,ббт
13,абвер,s,2.7,22,38,25,бвр
16,абзац,s,10.2,99,87,478,бзц
24,аборт,s,8.3,93,87,300,брт
...,...,...,...,...,...,...,...
52056,ямщик,s,1.4,50,81,74,мщк
52119,ятаган,s,0.6,24,75,37,тгн
52132,ящер,s,2.0,51,73,63,щр
52133,ящерица,s,3.6,77,74,158,щрц


## Поиск оптимума

Далее мы должны создать функцию, которая будет сопоставлять букве цифру. Вполне вероятно, что в лучшие комбинации достигаются при распределении отличном от парного разбиения.

При рассмотрении соответствия всех согласных получится $10^{21}$, что явно бессмысленно для нашей задачи. Если не утяжелять задачу, можно просто использовать случайные последовательности и максимизировать параметр уникальности получаемых чисел.

In [202]:
import numpy as np
digs = np.random.randint(0, 10, 21)
trans_table = {c: str(d) for (c, d) in zip (cons, digs)}
trans_table

{'б': '6',
 'в': '6',
 'г': '6',
 'д': '8',
 'ж': '1',
 'з': '1',
 'й': '6',
 'к': '7',
 'л': '1',
 'м': '9',
 'н': '0',
 'п': '9',
 'р': '3',
 'с': '8',
 'т': '9',
 'ф': '3',
 'х': '1',
 'ц': '8',
 'ч': '3',
 'ш': '3',
 'щ': '0'}

Напишем две функции:
1) функцию преобразования слова к запоминаемой последовательности цифр,
2) функцию, результат которой мы хотим максимизировать

In [199]:
def translate(word):
    digit_symbols = []
    for letter in word:
        digit_symbols.append(trans_table.get(letter))
    return "".join(digit_symbols)

{'б': 6,
 'в': 5,
 'г': 6,
 'д': 8,
 'ж': 9,
 'з': 7,
 'й': 9,
 'к': 3,
 'л': 0,
 'м': 2,
 'н': 0,
 'п': 8,
 'р': 8,
 'с': 9,
 'т': 0,
 'ф': 3,
 'х': 1,
 'ц': 4,
 'ч': 6,
 'ш': 9,
 'щ': 6}